In [1]:
import pandas as pd

# 교육 관련 데이터 제공 (교육 통계 서비스)
# https://kess.kedi.re.kr/post/6684877?itemCode=04&menuId=m_02_04_02&code=&words=%EC%A3%BC%EC%86%8C%EB%A1%9D
# 대학교 리스트를 데이터프레임 변환
df = pd.read_excel('./고등교육기관 주소록_2019.xlsx', header=5)
df.head()

,연도,학교종류,시도,학교명,학교명(영문),본분교,학교상태,설립,우편번호,주소,전화번호,팩스번호,홈페이지
0,2019,전문대학(4년제),서울,서울여자간호대학교,Seoul Women's College of nursing,본교,기존,사립,3617,"서울특별시 서대문구 간호대로 38 (홍제동, 서울여자간호대학)",02-2287-1700,02-395-8018,www.snjc.ac.kr
1,2019,전문대학(4년제),광주,기독간호대학교,Christian College of Nursing,본교,기존,사립,61662,"광주광역시 남구 백서로70번길 6 (양림동, 기독간호대학)",062-650-8000,062-675-5806,ccn.ac.kr
2,2019,전문대학(4년제),광주,조선간호대학교,CHOSUN NURSING COLLEGE,본교,기존,사립,61453,"광주광역시 동구 필문대로 309-2 (서석동, 조선간호대학교)",062-231-7022,062-231-7029,www.cnc.ac.kr
3,2019,전문대학(4년제),전북,군산간호대학교,KUNSAN COLLEGE OF NURSING,본교,기존,사립,54068,"전라북도 군산시 동개정길 7 (개정동, 군산간호대학교)",063-450-3855,063-450-3899,www.kcn.ac.kr
4,2019,전문대학(3년제),서울,동양미래대학교,Dongyang Mirae University,본교,기존,사립,8221,"서울특별시 구로구 경인로 445 (고척동, 동양미래대학교)",02-2610-1700,02-2688-5494,www.dongyang.ac.kr/


In [6]:
df_1 = df[(df['연도'] == 2019) & (df['시도'] == '부산')].loc[:, ['학교명', '주소']]
df_1.head()

,학교명,주소
9,경남정보대학교,"부산광역시 사상구 주례로 45 (주례동, 경남정보대학)"
10,동부산대학교,"부산광역시 해운대구 운봉길 60 (반송동, 동부산대학교)"
11,동주대학교,"부산광역시 사하구 사리로55번길 16 (괴정동, 동주대학)"
79,대동대학교,"부산광역시 금정구 동부곡로27번길 88 (부곡동, 대동대학교)"
80,동의과학대학교,"부산광역시 부산진구 양지로 54 (양정동, 동의과학대학)"


In [7]:
print(len(df_1))

28


In [8]:
# https://developers.kakao.com/docs/restapi/local#%EC%A3%BC%EC%86%8C-%EA%B2%80%EC%83%89
import requests
KAKAO_API_KEY = '5175c0a009281f3d39afe4b7dcaa6d3d'
url  = '''
https://dapi.kakao.com/v2/local/search/address.json?query={0}
'''.format( '부산광역시 사상구 주례로 45 (주례동, 경남정보대학)' )
headers = {
    'Authorization':'KakaoAK {0}'.format(KAKAO_API_KEY)
}
res = requests.get( url , headers=headers )
res = res.json()
print ( res['documents'][0]['address_name'],
        res['documents'][0]['y'],
        res['documents'][0]['x'])

부산 사상구 주례로 45 35.14645115598116 129.009068796574


In [9]:
import requests
KAKAO_API_KEY = '5175c0a009281f3d39afe4b7dcaa6d3d'
def netGetGeocoder(  query ):
    url  = '''
    https://dapi.kakao.com/v2/local/search/address.json?query={0}
    '''.format( query )
    headers = {
        'Authorization':'KakaoAK {0}'.format(KAKAO_API_KEY)
    }
    # API 쿼리 수행
    res = requests.get( url , headers=headers )
    res = res.json()
    if len(res['documents']) > 0:
        # 데이터 리턴 : 주소, 위도, 경도, 질의어 순으로 튜플 형태로 리턴     
        return [res['documents'][0]['address_name'], res['documents'][0]['y'],res['documents'][0]['x']]
#     print(len(res['documents']))

In [11]:
from tqdm import tqdm_notebook

# 데이터를 담는 자료형
university_list = []
# df_2 = df_1.loc[:, ['주소', '학교명']]
# df_2.head()
# 경찰서 이름 개수만큼 반복
for i, addr in tqdm_notebook(df_1.iterrows()):
    result = netGetGeocoder( addr['주소']  )
    if result != None :
        result.append(addr['학교명'])
        university_list.append( result )

print(len(university_list))


28


In [12]:
df_2 = pd.DataFrame(university_list)
df_2.head()

,0,1,2,3
0,부산 사상구 주례로 45,35.14645115598116,129.009068796574,경남정보대학교
1,부산 해운대구 운봉길 60,35.23274266294026,129.14902810747094,동부산대학교
2,부산 사하구 사리로55번길 16,35.10672472289597,128.99723202457403,동주대학교
3,부산 금정구 동부곡로27번길 88,35.22913348950384,129.09586549584338,대동대학교
4,부산 부산진구 양지로 54,35.1658506524463,129.07222594898334,동의과학대학교


In [13]:
import folium

In [14]:
# 서울 지도 만들기
pusan_map = folium.Map(location=[35.1797445386017,129.075076791394], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 Marker로 표시
for university_info in university_list:
    folium.Marker([university_info[1], university_info[2]], popup=university_info[3]).add_to(pusan_map)

# 지도를 HTML 파일로 저장하기
pusan_map.save('./pusan_university3.html')